## Implementação de um Classificador Perceptron e Multilayer Perceptron

In [ ]:
import numpy as np


class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        y : array-like, shape = [n_examples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

## Testando o classificador Perceptron

In [ ]:
"""Dados de Treinamento """
X = np.array([[1,1],[2,2],[3,3]])
y = np.array([1,1,-1])

"""Criando objeto Perceptron"""
ppn = Perceptron(eta=0.1, n_iter=100)

"""Treinando o modelo"""
ppn.fit(X, y)

"""Testando modelo treinado """
X_newdata = np.array([[4,4],[2,2],[3,3]])
print("Resultado da Predição",ppn.predict(X_newdata));

Resultado da Predição [-1  1 -1]


## Questao 1 - Implementar o modelo Multilayer Perceptron (MLP) e testar este classificador

In [ ]:
### Modelo de regressão mlp com loss
import numpy as np

class MLP:
    def __init__(self, layers):
        self.layers = layers
        self.weights = []
        self.biases = []
        self.activations = []

        # Inicialização dos pesos e biases
        for i in range(len(layers) - 1):
            weight_matrix = np.random.randn(layers[i + 1], layers[i])
            bias_vector = np.zeros((layers[i + 1], 1))
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def feedforward(self, x):
        activation = x
        self.activations = [np.array(activation)]  # Convert to numpy array

        for i in range(len(self.weights)):
            weighted_input = np.dot(self.weights[i], activation) + self.biases[i]
            activation = self.sigmoid(weighted_input)
            self.activations.append(np.array(activation))  # Convert to numpy array

        return activation

    def softmax(x):
      exponent = np.exp(x) # only compute the exponent once
      return exponent/exponent.sum(axis=1,keepdims=True)

    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            for x, y in zip(x_train, y_train):
                # Forward propagation
                self.feedforward(x)

                # Backpropagation
                error = y - self.activations[-1]
                delta = error * self.activations[-1] * (1 - self.activations[-1])

                for i in range(len(self.weights) - 1, -1, -1):
                    gradient = learning_rate * np.dot(delta, self.activations[i].T)
                    self.weights[i] += gradient
                    self.biases[i] += learning_rate * np.mean(delta, axis=1, keepdims=True)

                    error = np.dot(self.weights[i].T, delta)
                    delta = error * self.activations[i] * (1 - self.activations[i])

    def predict(self, x):
        return self.feedforward(x)


In [1]:
import numpy as np

class MLP:
    def __init__(self, layers):
        self.layers = layers
        self.weights = []
        self.biases = []
        self.activations = []

        # Inicialização dos pesos e biases
        for i in range(len(layers) - 1):
            weight_matrix = np.random.randn(layers[i + 1], layers[i])
            bias_vector = np.zeros((layers[i + 1], 1))
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exponent = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exponent / np.sum(exponent, axis=1, keepdims=True)

    def feedforward(self, x):
        activation = x
        self.activations = [np.array(activation)]  # Convert to numpy array

        for i in range(len(self.weights)):
            weighted_input = np.dot(self.weights[i], activation) + self.biases[i]
            activation = self.sigmoid(weighted_input)
            self.activations.append(np.array(activation))  # Convert to numpy array

        return activation

    def cross_entropy_loss(self, y_pred, y_true):
        epsilon = 1e-10
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.sum(y_true * np.log(y_pred)) / len(y_pred)
        return loss

    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            for x, y in zip(x_train, y_train):
                # Forward propagation
                output = self.feedforward(x)
                y_pred = self.softmax(output)

                # Backpropagation
                error = y_pred - y
                delta = error

                for i in range(len(self.weights) - 1, -1, -1):
                    gradient = learning_rate * np.dot(delta, self.activations[i].T)
                    self.weights[i] -= gradient
                    self.biases[i] -= learning_rate * np.mean(delta, axis=1, keepdims=True)

                    error = np.dot(self.weights[i].T, delta)
                    delta = error

    def predict(self, x):
        return self.feedforward(x)


## Questao 2a - Implemente uma função para calcular a acurácia dos modelos

In [ ]:
mlp.train(x,y,5,0.5)

## Questao 2b - Implemente um método de validação cruzada para testar

## Questao 2c - Teste os classificadores usando um conjunto de dados linearmente separável e outro não linearmente separável
### Sugestão: crie datasets sintéticos com apenas dois atributos para voce poder visualizar a separação das classes